In [1]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device}.')

Using cuda.


In [13]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn

In [3]:
# You don't need to understand this function for now.
def load_data_CIFAR10(batch_size, resize=None):
    """Download the CIFAR10 dataset and then load it into memory."""
    trans = [torchvision.transforms.ToTensor()]
    if resize:
        trans.insert(0, torchvision.transforms.Resize(resize))
    trans = torchvision.transforms.Compose(trans)
    mnist_train = torchvision.datasets.CIFAR10(
        root="../data", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.CIFAR10(
        root="../data", train=False, transform=trans, download=True)
    return (torch.utils.data.DataLoader(mnist_train, batch_size, shuffle=True,
                            num_workers=2),
            torch.utils.data.DataLoader(mnist_test, batch_size, shuffle=False,
                            num_workers=2))

In [4]:
batch_size = 64 # Defines the batch size
train_iter, test_iter = load_data_CIFAR10(batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
X, y = next(iter(train_iter)) # Requests the first training batch
print(X.size()) # 256 images per batch. Each image is represented by a 1 x 28 x 28 tensor (number of channels x height x width). The images are grayscale, so there is a single channel.
print(y.size())

torch.Size([64, 3, 32, 32])
torch.Size([64])


In [6]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        
        self.fc = nn.Linear(out_channels, 1)
        
    def forward(self, x):
        out = self.relu(x)
        out1 = self.conv1(out)
        out2 = self.conv2(out)
        out3 = self.conv3(out)
        out4 = self.conv4(out)
        out5 = self.conv5(out)
        
        # Average pooling
        avg_pool = x.
        avg_pool = avg_pool.view(avg_pool.size(0), -1)
        # Weighted average calculation
        weights = torch.sigmoid(self.fc(avg_pool))
        print(weights.type())
        # Weighted sum
        weighted_sum = (out1 + out2 + out3 + out4 + out5) * weights.view(-1,1,1,1)
        
        return weighted_sum

In [8]:
# Applies Xavier initialization if the `torch.nn.Module` is `torch.nn.Linear` or `torch.nn.Conv2d`
def init_weights(m):
    if type(m) == torch.nn.Linear or type(m) == torch.nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)

num_outputs = 10
model = ConvBlock(in_channels=3, out_channels=num_outputs)
model.apply(init_weights) # Applies `init_weights` to every `torch.nn.Module` inside `model`

ConvBlock(
  (relu): ReLU()
  (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc): Linear(in_features=10, out_features=1, bias=True)
)

In [9]:
loss = torch.nn.CrossEntropyLoss()

In [10]:
lr = 0.9
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [11]:
def correct(logits, y):
    y_hat = logits.argmax(axis=1) # Finds the column with the highest value for each row of `logits`.
    return (y_hat == y).float().sum() # Computes the number of times that `y_hat` and `y` match.

In [12]:
def evaluate_metric(model, data_iter, metric):
    """Compute the average `metric` of the model on a dataset."""
    c = torch.tensor(0.)
    n = torch.tensor(0.)
    for X, y in data_iter:
        logits = model(X)
        c += metric(logits, y)
        n += len(y)

    return c / n

In [ ]:
print(f'Training accuracy: {evaluate_metric(model, train_iter, correct)}. Testing accuracy: {evaluate_metric(model, test_iter, correct)}.')

# Basic Code

In [ ]:
o = i

In [12]:
n = torch.randn(3,6,6)
n

tensor([[[ 0.5916,  0.5676, -1.0415,  0.2531,  1.1415, -1.0663],
         [ 1.0914,  0.5812,  0.7329, -0.9620, -3.5427, -0.9107],
         [ 0.1481, -0.6351, -2.2436,  0.2195,  0.3712, -0.2928],
         [-0.3627, -0.1971, -1.1930,  0.7793, -0.3895,  1.8926],
         [ 0.6179,  1.7252, -1.0225, -0.3343, -0.7708, -1.1098],
         [-0.8818,  0.1688, -0.0045,  0.5856, -0.9097, -0.2356]],

        [[-0.9179,  0.8336, -0.8736, -0.5851,  0.9430,  0.1678],
         [ 1.1078,  0.6509,  0.2739,  0.2685,  0.1837, -0.4887],
         [ 0.1076, -1.4711, -0.1499,  1.0199,  1.2663,  1.3709],
         [-0.8724, -0.8335,  0.5277, -1.7724,  1.4742,  0.6399],
         [-0.3226,  0.7655,  0.9605, -0.1787,  0.4703, -1.9245],
         [-1.9839, -1.3509, -0.2614, -0.0918, -1.8255, -0.7549]],

        [[ 0.1341,  0.6445,  0.7236, -0.3718, -1.1775, -0.1703],
         [ 0.8173, -1.1339, -0.3453,  0.7949, -0.0057,  0.5891],
         [ 1.2043,  1.0888,  0.1014, -2.1153, -0.5242, -0.2361],
         [ 0.5098,  0

In [9]:
import torch
n = torch.Tensor([[[1,2,6,4,7],[2,4,3,5,7]],[[1,1,4,3,2],[2,2,7,5,7]], [[1,4,8,4,7],[2,4,8,5,7]]])
m = []
for i in range(n.size()[0]):
    m.append((float(n[i].flatten().mean())))
m = torch.Tensor(m)
m

tensor([4.1000, 3.4000, 5.0000])

In [3]:
class Model(torch.nn.Module):
    
    def __init__(self, ):
        super(Model, self).__init__()
        self.fc1 = torch.nn.Linear(3, 5)
    def forward(self, x):
        out  = self.fc1(x)
        return out

In [4]:
k = Model()
out = k(m)

NameError: name 'm' is not defined

In [5]:
class BlockGen(nn.Module):
    # Creating Intermediate Blocks
    # the length of param indicates the number of intermediate blocks
    # each parameter is then inside the param for convolutional layers
    def __init__(self, in_channels, out_channels, c, layers, paddings, strides, kernels):
        super(BlockGen, self).__init__()
        self.relu = nn.ReLU()
        convolutions = nn.ModuleList
        
        for i in c:
            convs = nn.ModuleList()
            for j in range(layers[i]):
                convs.append(nn.Conv2d(in_channels[i][j], out_channels[i][j], kernel_size=kernels[i][j],stride= strides[i][j], padding= paddings[i][j]))
            covolutions.append(convs)
        self.model = convolutions
        # the fully-connected layer turning m to a to calculate the weights of convolutional layer in the final equation
        self.a = nn.Linear(in_channels, len(param))
        
    def forward(self, x):
        m = []
        for i in range(x.size()[0]):
            m.append((float(x[i].flatten().mean())))
        m = torch.Tensor(m)
        weights = self.a(m)
        # creating the convolutional neurons in the layer of the block
        x = self.relu(x)
        layer = []
        for conv in self.model:
            layer.append(conv(x))
        return layer
        

In [6]:
class OutBlock:

SyntaxError: incomplete input (2093479193.py, line 1)

In [109]:
param = [{'L': 3,'kernel_size':1 , 'stride':1, 'padding':0}, {'out':3, 'L': 3,'kernel_size':2 , 'stride':3, 'padding':1}, {'out':2, 'L': 5,'kernel_size':2 , 'stride':1, 'padding':0}]

In [110]:
for i, j in enumerate(param):
    print(param[i])

{'L': 3, 'kernel_size': 1, 'stride': 1, 'padding': 0}
{'out': 3, 'L': 3, 'kernel_size': 2, 'stride': 3, 'padding': 1}
{'out': 2, 'L': 5, 'kernel_size': 2, 'stride': 1, 'padding': 0}


In [8]:
mod

NameError: name 'mod' is not defined

In [7]:
mod = Blo"ckGen(param=param, in_channels=3, out_channels=10)

SyntaxError: unterminated string literal (detected at line 1) (4252390199.py, line 1)

In [113]:
out = mod(n)
len(out)

3

In [100]:
len(param)

3

In [ ]:
mo = nn.Conv2d()